In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar100
import numpy as np
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import time

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Select 20 classes
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

# Remap labels to 0-19
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

# One-hot encode labels
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

# Resize images to 224x224
x_train_20_resized = tf.image.resize(x_train_20, (224, 224)).numpy()
x_test_20_resized = tf.image.resize(x_test_20, (224, 224)).numpy()

# Preprocess images
x_train_20_resized = preprocess_input(x_train_20_resized)
x_test_20_resized = preprocess_input(x_test_20_resized)

# List of model constructors and names
model_constructors = [
    (Xception, "Xception"),
    (ResNet50, "ResNet50"),
    (MobileNet, "MobileNet"),
    (MobileNetV2, "MobileNetV2"),
    (DenseNet121, "DenseNet121"),
    (NASNetMobile, "NASNetMobile"),
    (EfficientNetB0, "EfficientNetB0"),
    (EfficientNetB1, "EfficientNetB1"),
    (EfficientNetB3, "EfficientNetB3"),
    (InceptionV3, "InceptionV3")
]

# Loop through each model
for model_fn, model_name in model_constructors:
    print(f"\nTraining {model_name}...")

    input_tensor = Input(shape=(224, 224, 3))
    base_model = model_fn(weights='imagenet', include_top=False, input_tensor=input_tensor)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(20, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    model.fit(x_train_20_resized, y_train_20_cat, epochs=1, batch_size=32, verbose=1)
    end_time = time.time()

    loss, accuracy = model.evaluate(x_test_20_resized, y_test_20_cat, verbose=0)
    print(f"{model_name} - Test accuracy: {accuracy:.4f}, Time taken: {end_time - start_time:.2f} seconds")


: 